In [ ]:
import tess_cpm

In [ ]:
fits_file = "tpf/tess-s0001-4-1_64.525833_-63.615669_64x64_astrocut.fits"

In [ ]:
cpm = tess_cpm.CPM(fits_file, remove_bad=True)

In [ ]:
cpm.set_target_exclusion_predictors(32, 32)

In [ ]:
cpm.lsq(0.5, rescale=True, polynomials=True)

In [ ]:
tess_cpm.summary_plot(cpm, 10)

In [ ]:
cpm.entire_image(0.5, polynomials=True)

In [ ]:
tess_cpm.summary_plot(cpm, 10)

In [ ]:
apt, apt_lc = cpm.difference_image_sap(32, 32, 1)

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(cpm.time, apt_lc)

# apt = apt.reshape(apt.shape[0], 9)
# for i in range(9):
#     plt.plot(cpm.time, apt[:, i])
# # plt.plot(cpm.time, apt_lc, ".", color="Red")

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [ ]:
from astroquery.mast import Tesscut

In [ ]:
c = SkyCoord(64.525833, -63.615669, unit='deg')

In [ ]:
c.dec

In [ ]:
tesscut_test_download = Tesscut.download_cutouts(c)

In [ ]:
c = SkyCoord("5:33:05.28 -69:30:31.44", unit=(u.hourangle, u.deg))

In [ ]:
moa = Tesscut.download_cutouts(c, size=64, sector=2)

In [ ]:
fits_file = "./tess-s0002-4-2_83.272000_-69.508733_64x64_astrocut.fits"
c = tess_cpm.CPM(fits_file, remove_bad=True)

In [ ]:
c.set_target_exclusion_predictors(32, 32)

In [ ]:
c.lsq(0.1, rescale=True, polynomials=True)

In [ ]:
tess_cpm.summary_plot(c, 10)

In [ ]:
c.entire_image(0.1, polynomials=True)

In [ ]:
apt, apt_lc = c.difference_image_sap(32, 32, 2)

In [ ]:
plt.figure(figsize=(15 ,10))
apt = apt.reshape(apt.shape[0], 25)
for i in range(25):
    plt.plot(c.time, apt[:, i])
plt.plot(c.time, apt_lc, ".", color="Red")

In [ ]:
from IPython.display import HTML
import matplotlib.animation as animation
import numpy as np
diff = c.im_diff

fig = plt.figure(figsize=(10, 10))

ims = []
for i in range(0, diff.shape[0], 5):
    im = plt.imshow(diff[i], origin="lower", animated=True,
                   vmin=np.percentile(diff, 1), vmax=np.percentile(diff, 99));
    ims.append([im]);
fig.colorbar(im)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000);

HTML(ani.to_jshtml())

In [ ]:
c.set_target_exclusion_predictors(55, 40)
c.lsq(0.3, polynomials=False)
tess_cpm.summary_plot(c, 10)

In [ ]:
c.set_target_exclusion_predictors(55, 40)
c.lsq(0.3, polynomials=True)
tess_cpm.summary_plot(c, 10)

In [ ]:
ap, lc = c.difference_image_sap(55, 40, 1)

In [ ]:
plt.figure(figsize=(15, 10))
ap = ap.reshape(ap.shape[0], 9)
for i in range(9):
    plt.plot(c.time, ap[:, i], '-')
plt.plot(c.time, lc)